In [1]:
import requests
from lxml import etree
import re
import sys
import parsel
headers = {
    #填自己的Cookie
    "Cookie": "fspop=test; _lxsdk_cuid=1789acb1ea1c8-0799a204fdb9ea-5c3f1e49-10ed98-1789acb1ea1c8; _lxsdk=1789acb1ea1c8-0799a204fdb9ea-5c3f1e49-10ed98-1789acb1ea1c8; _hc.v=ebb31d04-778f-02a6-9984-060d56d71344.1617504706; s_ViewType=10; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; cy=79; cye=haerbin; ctu=3a87482384aa8445669918ae73680ee9a42215c28654ba91ce2d4ec9bf99ab35; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1617504706,1617504873,1617692904; ll=7fd06e815b796be3df069dec7836c3df; ua=dpuser_9453298353; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1617706392; _lxsdk_s=178a674feaa-17a-da3-270%7C%7C1311",
    "Host": "www.dianping.com",
    "Referer": "http://www.dianping.com/shop/l1MNeHgCskgs6qiN",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

headers1 = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

url = "http://www.dianping.com/shop/G6KeVfRgAOACdDbG/review_all"
response = requests.get(url=url,headers=headers)
response.encoding='UTF-8' #编码用utf-8
#保存爬取到的的html
with open(sys.path[0] + "/" + '大众点评.html','w',encoding='UTF-8') as f:
    f.write(response.text)
content = response.content
html = etree.HTML(content)
css_url = 'http:'+html.xpath('/html/head/link[4]/@href')[0]

#css的url
# print(css_url)
css_response = requests.get(url=css_url,headers=headers1)
response.encoding = 'windows-1252'
# print(css_response.text)
#这里我对 css网页也进行了保存
with open(sys.path[0] + "/" + r'大众点评.css','w',encoding='UTF-8') as f:
    f.write(css_response.text)
#这里xpath无法用，我们可以用正则
svg_group = re.search(r'svgmtsi\[class\^="(\w+)"].*?background-image: url\((.*?)\);', css_response.text)
#key_letter是为了获取属性class的值。
key_letter = svg_group[1]
svg_url = 'http:'+svg_group[2]
# print(key_letter)
# print(svg_url)
svg_response = requests.get(url=svg_url,headers=headers1)
with open(sys.path[0] + "/" + "大众点评.svg",mode='w',encoding='UTF-8') as f:
    f.write(svg_response.text)


with open(sys.path[0] + "/" + "大众点评.svg",'r',encoding='UTF-8') as f:
    svg_html = f.read()

list1=re.findall(r'id="(\d+)"\sd="M0\s(\d+) ',svg_html)
href_list=[i[0] for i in list1]
y_list=[]
for i in list1:
    y_list.append(str(int(i[1])-23))

dic = dict(zip(href_list, y_list))

sel = parsel.Selector(svg_html)
texts = re.findall(r'<textPath xlink:href="#(.*?)" textLength="(.*?)">(.*?)</textPath>',svg_html)
lines = {}

for text in texts:
    lines[int(text[0])] = text[2]

with open(sys.path[0] + "/" + '大众点评.css','r',encoding='utf-8') as f:
    css_html = f.read()

class_map = re.findall('\.('+key_letter+'\w+){background:-(\d+)\.0px -(\d+)\.0px;\}', css_html)
# print(class_map)
class_map = [(cls_name, int(x), int(y)) for cls_name, x,y in class_map]

resDic = {}
for one_char in class_map:
    cls_name, x, y = one_char
    for d in dic.keys():
        if y == int(dic[d]):
            index = (int)(x / 14)
            resDic[cls_name] = lines[int(d)][index]
            break
with open(sys.path[0] + "/" + '大众点评.html',mode="r",encoding="UTF-8") as f:
    html = f.read()

comment_list=re.findall('<div class="review-words Hide">\s+(.*?)\s+<div class="less-words">',html)
for i in comment_list:
    key_list=re.findall('<svgmtsi class="(\w+)"></svgmtsi>',i)
    for n in key_list:
        #这里是通过循环把类似<svgmtsi class="izu41"></svgmtsi>的替换相应的汉字
        i = i.replace('<svgmtsi class="{}"></svgmtsi>'.format(n),resDic[n])

        #这里这里img_list是为了把那些评论的表情包删掉。)
        i = re.sub(r'<img (.*?)/>',"",i)
        i = re.sub(r'&(.*?);',"",i)
    print(i + "\n")


入口即化好吃到爆炸！不愧是京城第一和牛烤肉，从食材品质到服务意识，全部在线✈️位置：按导航到即可，但是周边停车位不富裕，建议打车前往🌿环境：日式装潢，全体服务人员身穿和服，瞬间带入仿佛置身于日本，店里还有个网红气球房，但是感觉氛围没有楼下好，就果断放弃了🥩食材：真的几乎每一样都不踩雷，由专业烧肉师烤的和牛肉入口即化，火候掌握的恰到好处，由于肉过于鲜美，导致蘸料简直就是多余的存在。虽然之前看过攻略推荐，但还是在种类繁多的菜单上挑花了眼，所以俩人干脆直接点了个和牛拼盘，下次人多再点现切m12好了「十勝名物和牛五种拼盘」如果你也在菜单里挑花了眼，点这个准没错！每一种肉都不会踩雷，伴随着牛肉的鲜香每一口都要入口即化似的，而且不要看才400g，两个人吃刚刚好，我就眼大肚子小的以为不够，还点了个黑猪肉结果没吃完「M8一级棒小排」口感偏瘦，但火候掌握的很好肉质鲜美，没有柴的口感「王牛舌」确实有被惊艳到，厚实的牛舌烤完之后还鲜嫩多汁！眼睛直接冒光了「土豆泥」玉米切片洒在土豆泥上，口感还可以「煮牛舌」牛舌入味，口感缠绵「牛米蔬菜拼盘」送的蔬菜拼盘啊，爽口解油腻，胡萝卜和黄瓜不够的话都可以跟服务生再要「盐烤银杏」有点老，口感差点意思「库巴饭」这个真是踩雷了，说他是麻辣烫拌饭又不是那味，反正不好吃不推荐「桃子嗨棒」「金宾冻结柠檬嗨棒」他家的嗨棒劲太大了「蜜桃味可尔必思」不错喝精光好吃是真的好吃！价格也是真***贵

早就听说这家排队排到怀疑人生的网红店，终于有机会来体验一次。对于没有提前预约的人来说，等位确实需要以小时来计算。不过美食果然没有辜负我的期望。他家食材的鲜嫩和美味真的让人流连忘返。推荐直接点他家的大拼或小拼。种类多，性价比高。当然高品质的M12和牛更值得尝尝。只有M12才能体会到唇齿间那种极致的嫩滑。另外他家服务也不错，服务人员全程烤制，不用自己动手。等位也有小吃赠送。如果想品尝正宗和牛，板前十勝是个好选择

鼓楼这家日料店是声名远扬的网红打卡店，二楼有露台梦幻包间，据说要提前预约。店里的服务员都特别热情，等位还有甜品、饮料可以品尝，非常周到。这家的「M9半熟牛肉富士山」比其他地方的要肥一些，应该是牛肉品级不同，众所周知，品级越好，牛肉脂肪分布有均匀约多一些。「M12壶渍蒜香牛肋条」没让人失望，烤制时的香味儿和火焰非常迷人。

在某平台被种草过这家日式烧肉店，因为鼓楼这里寸土